### S2.L4.e1 Structured Outputs using OpenAI API

###  Aplicación práctica en ecommerce

El sistema desarrollado podría integrarse directamente en un flujo de subida de productos de una tienda online:

- Al registrar un nuevo artículo, se genera automáticamente una descripción optimizada para SEO y un conjunto de palabras clave relevantes.
- Se asegura consistencia de formato (gracias a la validación con Pydantic).
- Se podría automatizar para múltiples idiomas o categorías.
- Permite escalar catálogos grandes reduciendo costes de redacción manual.

En un entorno real, este pipeline se integraría con el backend (por ejemplo, Flask o FastAPI) y guardaría los resultados validados en una base de datos antes de publicarlos en el CMS.


In [1]:
!pip install openai 

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 1.2 MB/s eta 0:00:01
   ------------------------------- -------- 0.8/1.0 MB 1.2 MB/s eta 0:00:01
   ------------------------------- -------- 0.8/1.0 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 796.8 kB/s  0:00:01

   ------------- -------------------------- 1/3 [distro]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ---

In [2]:
!pip freeze > ../../requirements.txt

0. Import required libraries

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from openai import OpenAI
import json 
import os 



1. Load api key from env

In [ ]:

api_key = os.getenv("MY_API_KEY")   # devuelve None si no existe
if not api_key:
    raise RuntimeError("Falta MY_API_KEY en el entorno")

client = OpenAI(api_key = api_key)


2. Defining Schema

In [ ]:
class SEOProduct(BaseModel):

    name : str = Field(...,description = "Nombre del producto")
    description: str = Field(...,description = " Descripción optimizada para SEO")
    keywords: list[str] = Field(..., min_items=3, max_items=10, description = "Palabras clave relevantes")
    seo_score : float = Field(...,ge =0,le = 100, description="Puntuación SEO del 0 al 100")

In [7]:
class Metadata(BaseModel):
    generation_date: str = Field(..., description = "Fecha de generación del contenido")
    model_used: str = Field(...,description="modelo de IA utilizado")
    

In [ ]:
#  Clase principal que combina producto y metadatos
class SEOResponse(BaseModel):
    product: SEOProduct
    metadata: Metadata


3. API call

In [ ]:
#  Prompt y llamada a la API con Structured Output
prompt = """
Generate a structured JSON response describing a fictitious product optimized for SEO.
The output must include product name, SEO description, keywords, SEO score, and metadata.
The product is: "Smart Fitness Watch Pro"
"""

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a helpful assistant generating structured SEO product data."},
        {"role": "user", "content": prompt},
    ],
    response_format={"type":"json_object"}
)

print(response)


In [ ]:
# ✅ Validación del JSON recibido contra el schema
try:
    validated = SEOResponse.model_validate(response)
    print(json.dumps(validated.model_dump(), indent=2, ensure_ascii=False))
except ValidationError as e:
    print("Error de validación:", e)


In [ ]:
# 🔁 Varias pruebas con diferentes productos
products = [
    "EcoSmart Blender X",
    "AI Cleaning Service",
    "UltraSound Toothbrush"
]

for prod in products:
    prompt = f"""
    Generate structured SEO data for the product: "{prod}"
    Return a valid JSON with 'product' and 'metadata' keys.
    The SEO score must be between 0 and 10.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating SEO product data in JSON format."},
            {"role": "user", "content": prompt},
        ],
        response_format={"type": "json_object"}
    )

    data = json.loads(response.choices[0].message.content)

    try:
        validated = SEOResponse.model_validate(data)
        print(f"\n🛒 Producto: {prod}")
        print(json.dumps(validated.model_dump(), indent=2, ensure_ascii=False))
    except ValidationError as e:
        print(f"❌ Error de validación para {prod}:", e)
